### Some tests for an Ising model implementation and visualization

The Ising model Hamiltonian is $\mathcal{H} = -J\sum_{\langle i,j \rangle} \sigma_i \sigma_j - \sum_j h\sigma_j$. The partition function is $Z=\sum_{\{\sigma_i\}} e^{-\beta U}$ with $U=-\epsilon\sum_{\langle i,j \rangle} \sigma_i \sigma_j$ (8.38 and 8.39). The partition function sums over each possible lattice states but there are $2^N$ possible lattice states so for a $10\times 10$ lattice, we have $2^{10^2}\approx 10^{30}$ configurations. This means writing a program that attempts to analyze each possible lattice state will not succeed.

The Metropolis algorithim is a method of importance sampling that generates sample satets for a Monte Carlo simulation (integration) with the correct Boltzmann probabilities. This is a natural choice because we should *probably* sample the most likely states the most --- we don't need to sample extremely rare states. (Think of choosing a more clever volume to integrate over instead of a rectangle to approximate the volume of some other shape). **Schroeder uses periodic (torus) BC**.

Why is Metropolis done 100 times per site? Here I implement the pseudo code in Schroeder.

In [1]:
import pygame
from pygame.locals import *
import numpy as np

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


I think I may be doing Metropolis incorrectly? It says 100 times per dipole, but this is not what Schroeder writes in the pseudocode, nor is it what I do here. Otherwise, the implemenation seems to work and agree with what he has.

In [9]:
import pygame
from pygame.locals import *
import sys
import numpy as np
import random

# Initializing Pygame
pygame.init()
 
# Initializing surface
surface = pygame.display.set_mode((500,500))
pygame.display.set_caption('Ising Simulation')
 
# Initializing lattice TODO: REWRITE TO DRAW COLOR BASED OFF SPIN???
spinUp = (250, 199, 72) #yellow
spinDown = (29, 47, 111) #blue
def initLattice(latticeSize: int, siteSize: int) -> list[list[int]]:
    lattice = np.zeros((latticeSize, latticeSize))
    for x in np.arange(0, latticeSize*siteSize, siteSize):
        for y in np.arange(0, latticeSize*siteSize, siteSize):
            spin = random.randint(0,1)
            if(spin == 1):
                lattice[int(x/siteSize)][int(y/siteSize)] = 1
                color = spinUp
            else:
                lattice[int(x/siteSize)][int(y/siteSize)] = -1
                color = spinDown
            pygame.draw.rect(surface, color, pygame.Rect(x, y, siteSize, siteSize)) 
    return lattice

# Update the color of a given site.
def updateLatticeColors(i: int, j: int, lattice: list[list[int]], siteSize: int) -> None:
    #if the current state is spin up, color the square spin up
    if(lattice[i][j] == 1):
        pygame.draw.rect(surface, spinUp, pygame.Rect(i*siteSize, j*siteSize, siteSize, siteSize))
        return
    #otherwise color spin down
    if(lattice[i][j] == -1):
        pygame.draw.rect(surface, spinDown, pygame.Rect(i*siteSize, j*siteSize, siteSize, siteSize))
        return
    #if neither case caught, there is an error
    print("Couldn't update lattice color at (%s,%s)" % (i,j)) #TODO: report at what coordinate the lattice could not update the color

# Calculate change in energy of lattice by flipping a single site (i,j)
def deltaU(i: int, j: int, lattice: list) -> float:
    '''
    This calulation requires considering neighboring sites (first term in Hamiltonian)
    Therefore, we will use periodic boundary conditions (torus)
    I would like to imlement the external field term so you can drive the system to specific states

    E1 = -spin(i,j)*sum(spin(neighbors)),     E2 = spin(i,j)*sum(spin(neighbors))
    Ediff = E2 - E1 = 2spin(i,j)*sum(neighbors) (if spin(i,j) is 1 (up))            <<<<< NO epsilon/J? unclear why, currently just implementing pseudocode exactly as written

    In the mean field approximation E_up = -4J*sum(spin(neighbors))/4) 

    i is vertical, j is horizontal, zero indexed
    '''

    size = len(lattice)
    # If site is in an edge, apply periodic boundary conditions
    if(i == 0):
        top = lattice[size-1,j]
    else:
        top = lattice[i-1][j]
    if(i == size-1):
        bottom = lattice[0][j]
    else:
        bottom = lattice[i+1][j]
    if(j == 0):
        left = lattice[i][size-1]
    else:
        left = lattice[i][j-1]
    if(j == size-1):
        right = lattice[i][0]
    else:
        right = lattice[i][j+1]

    #now calculate the energy difference
    Ediff = 2*lattice[i][j]*(top+bottom+left+right)
    return Ediff


size = 50
siteSize = 10
lat = initLattice(size,siteSize)
pygame.display.flip()

# Simulation loop
T = 1
while True:
     #in units of J/k (J is epsilon the coupling constant, k is boltzmann k) Tc=2.27

    #unclear if i should be doing 100 attempts of metropolis per site??? the pseudo code doesnt do this, it picks a new random site each time
    for iteration in range(100):
        #redraw lattice
        pygame.display.flip()

        #calc Ediff for a site
        i = random.randint(0,size-1) #draw from top left so 49 goes to 50
        j = random.randint(0,size-1)
        Ediff = deltaU(i,j,lat)
        #print("Ediff: %s" % Ediff)

        #Metropolis to decide whether site should be flipped
        if(Ediff <= 0):
            lat[i][j] = -lat[i][j] #need to update color here
            updateLatticeColors(i,j,lat,siteSize)
        else:
            #now only flip site according to Boltzmann factor
            boltzmannRandom = random.uniform(0,1)
            if(boltzmannRandom < np.exp(-Ediff/T)):
                lat[i][j] = -lat[i][j]
                updateLatticeColors(i,j,lat,siteSize)

    #Check if the user tries to close the window
    for event in pygame.event.get():
        #pygame.display.flip() UNCOMMENT THIS FOR COOL EFFECT

        #Change temp with mouse scroll
        if event.type == pygame.MOUSEWHEEL:
            if(event.y == 1):
                T += 0.1
            if(event.y == -1):
                T -= 0.1
            if(T < 0):
                T = 0
            print("T = %s" % T)

        # check if the event is the X button
        if event.type==pygame.QUIT:
            # if it is quit the game
            pygame.display.quit()
            pygame.quit()
            sys.exit()

T = 0.9
T = 0.8
T = 0.7000000000000001
T = 0.6000000000000001
T = 0.5000000000000001
T = 0.40000000000000013
T = 0.30000000000000016
T = 0.20000000000000015
T = 0.10000000000000014
T = 1.3877787807814457e-16
T = 0


C:\Users\Noah\AppData\Local\Temp\ipykernel_10960\4008691590.py:111: RuntimeWarning: divide by zero encountered in scalar divide
  if(boltzmannRandom < np.exp(-Ediff/T)):


T = 0.1
T = 0.2
T = 0.1
T = 0.2
T = 0.30000000000000004
T = 0.4
T = 0.5
T = 0.6
T = 0.7
T = 0.7999999999999999
T = 0.8999999999999999
T = 0.9999999999999999
T = 1.0999999999999999
T = 1.2
T = 1.3
T = 1.4000000000000001
T = 1.5000000000000002
T = 1.6000000000000003
T = 1.7000000000000004
T = 1.8000000000000005
T = 1.9000000000000006
T = 2.0000000000000004
T = 2.1000000000000005
T = 2.2000000000000006
T = 2.3000000000000007
T = 2.400000000000001
T = 2.500000000000001
T = 2.600000000000001
T = 2.700000000000001
T = 2.800000000000001
T = 2.9000000000000012
T = 3.0000000000000013
T = 3.1000000000000014
T = 3.2000000000000015
T = 3.3000000000000016
T = 3.4000000000000017
T = 3.5000000000000018
T = 3.600000000000002
T = 3.700000000000002
T = 3.800000000000002
T = 3.900000000000002
T = 4.000000000000002
T = 4.100000000000001
T = 4.200000000000001
T = 4.300000000000001
T = 4.4
T = 4.5
T = 4.6
T = 4.699999999999999
T = 4.799999999999999
T = 4.899999999999999
T = 4.999999999999998
T = 5.099999999

SystemExit: 

Next up would be doing MC with this to generate order parameter plots like with percolation.